In [1]:
import os
import json
from pprint import pprint
from copy import deepcopy

import numpy as np
import pandas as pd
import numpy as np
import json
import glob
import sys

sys.path.insert(0, "../")

root_path = '/Users/u6066091/Desktop/kaggle/input/corona_challenge/'

corona_features = {"doc_id": [None], "source": [None], "title": [None],
                  "abstract": [None], "text_body": [None]}
corona_df = pd.DataFrame.from_dict(corona_features)

json_filenames = glob.glob(f'{root_path}/**/*.json', recursive=True)

In [2]:
root = '/Users/u6066091/Desktop/kaggle'

In [14]:
root = '/Users/u6066091/Desktop/kaggle'
Design_list = [ 'retrospective cohort', 'cross-sectional case-control','cross sectional case control',
               'prevalence survey', 'systematic review ',' meta-analysis',' meta analysis'
               'matched case-control', 'matched case control','medical record review', 'observational case series', 
               'time series analysis','pseudo-randomized controlled trials' ,'pseudo randomized controlled trial',
               'randomized controlled trials','randomized controlled trial']
risk_factors = pd.read_csv(root + '/input/kaggle_annotated/COVID-19 medical_dictionary-difference.dictionary.csv')
cov_list = ['covid-19','covid 19', 'novel coronavirus','cord-19' ,'cord 19','2019-nCoV', 
            '2019-ncov','wuhan coronavirus']
risk_fac = ['diabete',
 'hypertension',
 'heart disease',
 'cancer',
 'smoking',
 'history of lung',
 'local climate',
 'elderly',
 'children',
 'immune compromised ',
 'age deciles',
 'race',
 'ethnicity',
 'education',
 'income',
 'insurance',
 'housing',
 'immigration',
 'prison inmate',
 'mental hospital inpatients',
 'long-term care facility resident',
 'long term care'           
 'health worker',
 'first responder',
 'hospital staff',
 'nursing home',
 'prison',
  'staff' ,                                          
 'pregnancy',
  'baby',
 'enfant',
 'pulm',
 'neonates']
Design_list.extend(['retrospective analysis','retrospective study','retrospective studies'])

In [16]:
#find section with design_study exists in text 


import re
import json
des_sec = []
for i in range(0,len(json_filenames)):
    
    with open(json_filenames[i], "r") as infile:
        filetext = infile.read()
    with open(json_filenames[i]) as data:
        data = json.load(data)      
        
        pattern = r'"text": "([^"]*(%s)[^"]*)((?!"section").)*"section": "([^"]*)"' % "|".join(Design_list)
        regex = re.compile(pattern, re.DOTALL)
   
        results = regex.findall(filetext.lower())
        for match in results:
            des_sec.append([data['paper_id'], match[3], match[1], match[0]])
#             print ('Match in section "%s":\n %s \n-------------------' % (match[3], match[0]))

In [18]:
design_sections = pd.DataFrame(des_sec, columns = ['doc_id', 'section', 'design_study', 'text'])
sec = [sec.lower() for sec in design_sections['section']]

In [19]:
#check for risk factors exists in text 


sec_risk = []
for i in range(0, len(design_sections['text'])):
    text = design_sections.loc[i,'text']
    risk = risk_fac
    ri_list = [re.findall(ri.lower(), text) for ri in risk]
    for ri in ri_list:
        if ri != []:
            sec_risk.append([design_sections.loc[i,'doc_id'],ri,text,design_sections.loc[i,'section'], design_sections.loc[i,'design_study']]
                           
                           
                           
                           )

In [26]:
risk_data = pd.DataFrame(sec_risk, columns =['doc_id', 'risk','text','section', 'design_study'])
set(risk_data['design_study'])

{' meta-analysis',
 'cross-sectional case-control',
 'matched case control',
 'medical record review',
 'prevalence survey',
 'randomized controlled trial',
 'randomized controlled trials',
 'retrospective analysis',
 'retrospective cohort',
 'retrospective studies',
 'retrospective study',
 'systematic review ',
 'time series analysis'}

In [31]:
# clean up

risk_data['design_study'].replace('retrospective analysis','retrospective cohort', inplace = True)
risk_data['design_study'].replace(['retrospective studies','retrospective study'],'retrospective cohort', inplace = True)
risk_data['design_study'].replace('randomized controlled trial','randomized controlled trials', inplace = True)

set(risk_data['design_study'])

{' meta-analysis',
 'cross-sectional case-control',
 'matched case control',
 'medical record review',
 'prevalence survey',
 'randomized controlled trials',
 'retrospective cohort',
 'systematic review ',
 'time series analysis'}

In [32]:
risk_doc = pd.DataFrame(sec_risk, columns = ['doc_id', 'risk','text','section', 'design_study'])
risk_doc.to_csv('/Users/u6066091/Desktop/kaggle/output/dataframe/risk_design.csv')

In [63]:
risk_doc.shape

(3069, 5)

In [62]:
risk_doc.groupby(['doc_id']).count()

,risk,text,section,design_study
doc_id,,,,
001b8a3bcea5b3a810fd9d03a1bc51cf708fe399,1,1,1,1
0023258070e16dc764316f6797bfd03fea23ee07,1,1,1,1
0036b28fddf7e93da0970303672934ea2f9944e7,4,4,4,4
0079559ab8717d83bc3df6be8533ff9672bb4367,2,2,2,2
007e325b2514afd152c8e396b96df06d05147dfd,1,1,1,1
...,...,...,...,...
ff78ee43150a59344d0a0cd68298c02032b43991,1,1,1,1
ffa807816bc71d1680c06f3127f477e3f3ae8708,2,2,2,2
ffae196fecc4da7f1897e22f8ce0dc0e03c8bcec,2,2,2,2


In [53]:
# check if COVID-19 exists in text
cov_data =[]
for i in range(0, len(risk_doc['text'])):
    text = risk_doc.loc[i,'text']
    cov_app = [re.findall(cov, text.lower()) for cov in cov_list]
    for cov in cov_app:
        if cov != []:
            cov_data.append([risk_doc.loc[i,'doc_id'], cov, risk_doc.loc[i,'risk'], risk_doc.loc[i,'text'],
                     risk_doc.loc[i,'section'],risk_doc.loc[i,'design_study']])
                     

In [92]:
#cleaning up

cov_section = pd.DataFrame(cov_data , columns = ['doc_id','covid19_in_text', 'risk_factor', 'text', 'section','design_study' ])

cov_section['risk_factor'] = [list(set(risk)) for risk in cov_section['risk_factor']]

cov_section['risk_factor'] = [x[0].replace('pulm','pulmonary disease') for x in cov_section['risk_factor']]

cov_section.to_csv('/Users/u6066091/Desktop/kaggle/output/dataframe/cov_risk_design.csv')




In [96]:
#merge the data with original data
root2 = '/Users/u6066091/Desktop/kaggle/output/corona_challenge/'
all_data = pd.read_csv(root2+'all_data_metadata.csv')

In [98]:
cov_section.merge(all_data, how ='left', left_on =['doc_id'], right_on =['doc_id'])

,doc_id,covid19_in_text,risk_factor,text,section,design_study,Unnamed: 0,source,title,abstract_x,...,pubmed_id,license,abstract_y,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,41e2dc42ae03a6fe3b55fda963ba7bc4a7683b3b,[novel coronavirus],pulmonary disease,ali/ards is thought to be the uniform expressi...,clinical associations,retrospective study,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d535e9201c038010f5cdad2faab7418d7fe520eb,[novel coronavirus],race,middle east respiratory syndrome coronavirus (...,introduction,retrospective analysis,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0fe76206c188469d89bd130e879094d11de2afdf,[novel coronavirus],staff,a recent meta-analysis of rcts of nutritional ...,a review of health care epidemiology articles ...,meta-analysis,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5241581d168b90832f88103bd40c8db2d843a6ea,"[novel coronavirus, novel coronavirus, novel c...",hospital staff,"in multiple in vitro systems, and was also eff...",acknowledgements,randomized controlled trial,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5241581d168b90832f88103bd40c8db2d843a6ea,"[novel coronavirus, novel coronavirus, novel c...",staff,"in multiple in vitro systems, and was also eff...",acknowledgements,randomized controlled trial,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1a12cc7d49d8521bec5d447ebb413cbcf5aca8f4,"[novel coronavirus, novel coronavirus]",staff,3.2.1 degree of suspicion that they were infec...,analysis of influencing factors of the mental ...,retrospective analysis,11559.0,BIORXIV,Study of the mental health status of medical p...,Objective: This paper studied the relationship...,...,NaN,medrvix,Objective: This paper studied the relationship...,2020-03-06,ning sun; jun xing; jun xu; ling shu geng; qia...,NaN,NaN,NaN,True,biorxiv_medrxiv
6,aafa6cdfe96a5cdaf7b7c2f04b11a5dbdd73b2df,[novel coronavirus],staff,the copyright holder for this preprint (which ...,discussion,meta-analysis,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,374a5295716649a80dca87097ad9ac902d344db6,"[novel coronavirus, novel coronavirus, novel c...",children,the copyright holder for this preprint (which ...,patient consent,randomized controlled trial,11843.0,BIORXIV,Outcome reporting from protocols of clinical t...,Databases of ICMJE-accepted clinical trial reg...,...,NaN,medrvix,Objectives: To examine heterogeneity of outcom...,2020-03-08,Ruijin Qiu; Xuxu Wei; Mengzhu Zhao; Changming ...,NaN,NaN,NaN,True,biorxiv_medrxiv
8,9ae476404f7ef1ec1ede965f0b898f31a5bf5a81,[novel coronavirus],staff,"in this retrospective study, 64 confirmed case...",abstract,retrospective study,11918.0,BIORXIV,"Epidemiological, Clinical Characteristics and ...","Backgrounds Since December 2019, a novel coron...",...,NaN,medrvix,"Backgrounds Since December 2019, a novel coron...",2020-03-13,Jie Liu; Liu Ouyang; Pi Guo; Hai sheng Wu; Pen...,NaN,NaN,NaN,True,biorxiv_medrxiv
9,9ae476404f7ef1ec1ede965f0b898f31a5bf5a81,[novel coronavirus],staff,despite the increased attention towards protec...,introduction,retrospective study,11918.0,BIORXIV,"Epidemiological, Clinical Characteristics and ...","Backgrounds Since December 2019, a novel coron...",...,NaN,medrvix,"Backgrounds Since December 2019, a novel coron...",2020-03-13,Jie Liu; Liu Ouyang; Pi Guo; Hai sheng Wu; Pen...,NaN,NaN,NaN,True,biorxiv_medrxiv
